# Generate Confusion Matrices

In [1]:
# Imports
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%config InlineBackend.figure_format = 'svg'
np.set_printoptions(edgeitems=10,linewidth=180)

## Load the technology and its settings

Below, choose which technology to load data and settings for:

In [2]:
# Choose technology here
TECH = 'A'

# Load settings for technology
with open(f"data/tech{TECH}/settings.json") as sfile:
    settings = json.load(sfile)

In [3]:
# Load data for technology
colnames = ["addr", "time", "r", "g", "gi", "range", "timept"]
data = pd.read_csv(f"data/tech{TECH}/relaxdata.min.tsv.gz", names=colnames, sep='\t')
data.head()

,addr,time,r,g,gi,range,timept
0,80000,0.0,337532.079408,0.000003,0.000003,0,0.0
1,80001,0.0,76320.104403,0.000013,0.000013,1,0.0
2,80002,0.0,48034.594014,0.000021,0.000021,2,0.0
3,80003,0.0,21027.150891,0.000048,0.000048,6,0.0
4,80004,0.0,33193.229603,0.000030,0.000030,4,0.0


In [4]:
# Specify the allocation
ranges = range(0, 32, 2) # Use every second range as the 4-bit allocation

# Clip to [0, gmax]
data["g"] = np.clip(data["g"], 0, settings["gmax"] * 30/32)
data["gi"] = np.clip(data["gi"], 0, settings["gmax"] * 30/32)

# Get the initial and final range at each timept
data["irange"] = np.int16((data["gi"] + settings["gmax"]/32)//(settings["gmax"]/16))
data["range"] = np.int16((data["g"] + settings["gmax"]/32)//(settings["gmax"]/16))

In [5]:
# Show some data
data.head(20)

,addr,time,r,g,gi,range,timept,irange
0,80000,0.0,337532.079408,0.000003,0.000003,0,0.0,0
1,80001,0.0,76320.104403,0.000013,0.000013,1,0.0,1
2,80002,0.0,48034.594014,0.000021,0.000021,1,0.0,1
3,80003,0.0,21027.150891,0.000048,0.000048,3,0.0,3
4,80004,0.0,33193.229603,0.000030,0.000030,2,0.0,2
5,80005,0.0,25751.131647,0.000039,0.000039,3,0.0,3
6,80006,0.0,19943.666308,0.000050,0.000050,3,0.0,3
7,80007,0.0,18814.811120,0.000053,0.000053,4,0.0,4
8,80008,0.0,16647.549300,0.000060,0.000060,4,0.0,4
9,80009,0.0,13469.155868,0.000074,0.000074,5,0.0,5


In [6]:
# Get confusion matrix
for timept in settings["times"]:
    confmat = data[data["timept"] == timept].groupby(["range", "irange"]).size().unstack(fill_value=0)
    print(timept)
    print(confmat)
    np.save(f"confmats/tech{TECH}/{timept}.npy", confmat)

0
irange   0     1     2     3     4     5     6     7    8     9     10    11  \
range                                                                          
0       499     0     0     0     0     0     0     0    0     0     0     0   
1         0  1035     0     0     0     0     0     0    0     0     0     0   
2         0     0  1030     0     0     0     0     0    0     0     0     0   
3         0     0     0  1043     0     0     0     0    0     0     0     0   
4         0     0     0     0  1057     0     0     0    0     0     0     0   
5         0     0     0     0     0  1021     0     0    0     0     0     0   
6         0     0     0     0     0     0  1010     0    0     0     0     0   
7         0     0     0     0     0     0     0  1011    0     0     0     0   
8         0     0     0     0     0     0     0     0  983     0     0     0   
9         0     0     0     0     0     0     0     0    0  1001     0     0   
10        0     0     0     0     0   